In [2]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import substring, col, when, sum as pyspark_sum, collect_list
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

24/10/25 18:36:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/25 18:36:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
locations = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "locations") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

locations.show()

+------------+-------------+
|       state|         city|
+------------+-------------+
|    New York|New York City|
|    New York|       Newark|
|    New York|      Buffalo|
|    New York|    Rochester|
|  California|San Francisco|
|  California|   Sacramento|
|  California|    San Diego|
|  California|  Los Angeles|
|       Texas|        Tyler|
|       Texas|       Temple|
|       Texas|       Taylor|
|       Texas|       Dallas|
|Pennsylvania| Philadelphia|
|Pennsylvania|   Pittsburgh|
|Pennsylvania|    Pottstown|
+------------+-------------+



In [4]:
# Step 1: Create 'match_checking' column
df_with_check = locations.withColumn(
    "match_checking", 
    when(substring(col("state"), 1, 1) == substring(col("city"), 1, 1), 1).otherwise(0)
)

# Step 2: Group by 'state', collect cities, and calculate matching count
result_df = df_with_check.groupBy("state").agg(
    collect_list("city").alias("cities"),  # Group and collect cities in a list
    pyspark_sum("match_checking").alias("matching_letter_count")  # Sum match_checking column
)

# Step 3: Filter for states with matching_letter_count > 0 and order by matching_letter_count in descending order
final_df = result_df.filter(col("matching_letter_count") > 0).orderBy(col("matching_letter_count").desc())

# Show the result
final_df.show(truncate=False)


+------------+-------------------------------------------+---------------------+
|state       |cities                                     |matching_letter_count|
+------------+-------------------------------------------+---------------------+
|Texas       |[Tyler, Temple, Taylor, Dallas]            |3                    |
|Pennsylvania|[Philadelphia, Pittsburgh, Pottstown]      |3                    |
|New York    |[New York City, Newark, Buffalo, Rochester]|2                    |
+------------+-------------------------------------------+---------------------+

